# Template for hackathon projects

This notebook simulates a generic ground experiment in a way that can be easily customized for your project

In [ ]:
import os
import sys

import healpy as hp
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import toast
import toast.pipeline_tools
from toast.mpi import MPI


# Are you using a special reservation for a workshop?
# If so, set it here:
nersc_reservation = None

# Load common tools for all lessons
import sys
sys.path.insert(0, "../lessons")
from lesson_tools import (
    check_nersc,
    fake_focalplane
)
nersc_host, nersc_repo, nersc_resv = check_nersc(reservation=nersc_reservation)

# Capture C++ output in the jupyter cells
%reload_ext wurlitzer

! [[ ! -e weather_Atacama.fits ]] && wget http://portal.nersc.gov/project/cmb/toast_data/example_data/weather_Atacama.fits

## Parameters

These arguments control the entire notebook

In [ ]:
class args:
    split_schedule = None
    schedule = "schedule.txt"
    sort_schedule = False  # Matters for parallelization
    weather = "weather_Atacama.fits"
    sample_rate = 100  # Hz
    # Noise parameters
    fknee = 1.0
    alpha = 2
    # Scanning parameters
    scan_rate = 120  # deg / s
    # half-wave plate
    hwp_rpm = None
    hwp_step_deg = None
    hwp_step_time_s = None
    fov = 3.0  # Field-of-view in degrees
    # Projection parameters
    coord = "C"
    nside = 64
    mode = "IQU"
    outdir = "maps"

## Observing schedule

We write the scheduler parameters to file.  These arguments produce one constant elevation scan but you can easily modify that.

In [ ]:
%%writefile schedule.par
--site-lat
-22.958064
--site-lon
-67.786222
--site-alt
5200
--site-name
Atacama
--telescope
LAT
--start
2020-01-01 00:00:00
--stop
2020-01-02 00:00:00
--el
60
--ces-max-time
86400
--out
schedule.txt

## Focalplane

In [ ]:
focalplane = toast.pipeline_tools.Focalplane(
    fake_focalplane(fov=args.fov, fknee=args.fknee, alpha=args.alpha),
    sample_rate=args.sample_rate, 
)

## TODGround to `toast.data`

In [ ]:
mpiworld, procs, rank = toast.mpi.get_world()
comm = toast.mpi.Comm(mpiworld)

# Load the observing schedule, append weather and focalplane to it
    
#schedules = toast.pipeline_tools.load_schedule(args, comm)
#toast.pipeline_tools.load_weather(args, comm, schedules)
# There could be more than one observing schedule, but not this time
#schedule = schedules[0]
#schedule.telescope.focalplane = focalplane

# Useful shorthands in what follows
telescope = 'GB'

# Create TODGround objects based on the entries in the schedule
    
data = toast.Data(comm)


In [ ]:
import numpy as np
from scipy.constants import degree
import healpy as hp
try:
    import ephem
except:
    ephem = None
from toast import qarray as qa
from toast.timing import function_timer, Timer
from toast.tod import Interval, TOD
from toast.healpix import ang2vec
from toast.todmap.pointing_math import quat_equ2ecl, quat_equ2gal, quat_ecl2gal
from toast.todmap.sim_tod import simulate_hwp


class TODGb(toast.todmap.TODGround):
    '''GB TOD
    '''
    @function_timer
    def __init__(
        self,
        mpicomm,
        detectors,
        samples,
        firsttime=0.0,
        rate=100.0,
        site_lon=0,
        site_lat=0,
        site_alt=0,
        el=60,
        scanrate=1,
        scan_accel=0.1,
        CES_start=None,
        CES_stop=None,
        sun_angle_min=90,
        sampsizes=None,
        sampbreaks=None,
        coord="C",
        report_timing=True,
        hwprpm=None,
        hwpstep=None,
        hwpsteptime=None,
        sinc_modulation=False,
        **kwargs
    ):
        if samples < 1:
            raise RuntimeError(
                "TODGround must be instantiated with a positive number of "
                "samples, not samples == {}".format(samples)
            )

        if ephem is None:
            raise RuntimeError("Cannot instantiate a TODGround object without pyephem.")

        if sampsizes is not None or sampbreaks is not None:
            raise RuntimeError(
                "TODGround will synthesize the sizes to match the subscans."
            )

        if CES_start is None:
            CES_start = firsttime
        elif firsttime < CES_start:
            raise RuntimeError(
                "TODGround: firsttime < CES_start: {} < {}"
                "".format(firsttime, CES_start)
            )
        lasttime = firsttime + samples / rate
        if CES_stop is None:
            CES_stop = lasttime
        elif lasttime > CES_stop:
            raise RuntimeError(
                "TODGround: lasttime > CES_stop: {} > {}" "".format(lasttime, CES_stop)
            )

        self._firsttime = firsttime
        self._lasttime = lasttime
        self._rate = rate
        self._site_lon = site_lon
        self._site_lat = site_lat
        self._site_alt = site_alt

        if el < 1 or el > 89:
            raise RuntimeError("Impossible CES at {:.2f} degrees".format(el))

        self._el = el * degree
        self._scanrate = scanrate * degree
        self._CES_start = CES_start
        self._CES_stop = CES_stop
        self._sun_angle_min = sun_angle_min
        if coord not in "CEG":
            raise RuntimeError("Unknown coordinate system: {}".format(coord))
        self._coord = coord
        self._report_timing = report_timing
        self._sinc_modulation = sinc_modulation

        self._observer = ephem.Observer()
        self._observer.lon = self._site_lon
        self._observer.lat = self._site_lat
        self._observer.elevation = self._site_alt  # In meters
        self._observer.epoch = ephem.J2000  # "2000"
        # self._observer.epoch = -9786 # EOD
        self._observer.compute_pressure()

        self._min_az = None
        self._max_az = None
        self._min_el = None
        self._min_el = None

        self._az = None
        self._commonflags = None
        self._boresight_azel = None
        self._boresight = None

        # Set the boresight pointing based on the given scan parameters

        tm = Timer()
        if self._report_timing:
            if mpicomm is not None:
                mpicomm.Barrier()
            tm.start()

        sizes, starts = self.simulate_scan(samples)

        if self._report_timing:
            if mpicomm is not None:
                mpicomm.Barrier()
            tm.stop()
            if (mpicomm is None) or (mpicomm.rank == 0):
                tm.report("TODGround: simulate scan")
            tm.clear()
            tm.start()

        self._fp = detectors
        self._detlist = sorted(list(self._fp.keys()))

        # call base class constructor to distribute data

        props = {
            "site_lon": site_lon,
            "site_lat": site_lat,
            "site_alt": site_alt,
            "el": el,
            "scanrate": scanrate,
            "scan_accel": scan_accel,
            "sun_angle_min": sun_angle_min,
        }
        super(toast.todmap.TODGround, self).__init__(
            mpicomm,
            self._detlist,
            samples,
            sampsizes=[samples],
            sampbreaks=None,
            meta=props,
            **kwargs
        )

        self._AU = 149597870.7
        self._radperday = 0.01720209895
        self._radpersec = self._radperday / 86400.0
        self._radinc = self._radpersec / self._rate
        self._earthspeed = self._radpersec * self._AU

        if self._report_timing:
            if mpicomm is not None:
                mpicomm.Barrier()
            tm.stop()
            if (mpicomm is None) or (mpicomm.rank == 0):
                tm.report("TODGround: call base class constructor")
            tm.clear()
            tm.start()

        self.translate_pointing()

        self.crop_vectors()

        if self._report_timing:
            if mpicomm is not None:
                mpicomm.Barrier()
            tm.stop()
            if (mpicomm is None) or (mpicomm.rank == 0):
                tm.report("TODGround: translate scan pointing")

        # If HWP parameters are specified, simulate and cache HWP angle

        simulate_hwp(self, hwprpm, hwpstep, hwpsteptime)

        return
    
    @function_timer
    def simulate_scan(self, samples):
        """ Simulate a constant elevation scan, either constant rate or
        1/sin(az)-modulated.

        """
        self._az = np.arange(samples)/self._rate*self._scanrate
        self._az %= 2 * np.pi
        self._min_el = self._el
        self._max_el = self._el
        self._commonflags = np.array([0]*samples, dtype=np.uint8)

        return [samples], [self._CES_start]

In [ ]:
import astropy.units as u

START_TIME = 1577839800.0
DURATION = 60*60*24
STOP_TIME = START_TIME + DURATION
SAMPLE_RATE = 100 #Hz

OT_ALTITUDE = 2390 # m
OT_LON = '{}'.format(-(17 + 29*1/60 + 16*1/60/60))
OT_LAT = '{}'.format(28 + 18*1/60 + 8*1/60/60)

totsamples = int((STOP_TIME - START_TIME)*SAMPLE_RATE)
todgb = TODGb(comm.comm_group,
              focalplane.detquats,
              totsamples,
              firsttime=START_TIME,
              el=60,
              site_lon=OT_LON,
              site_lat=OT_LAT,
              site_alt=OT_ALTITUDE,
              scanrate=120)
    
# Now embed the TOD in an observation dictionary and add other necessary metadata

obs = {}
obs["name"] = "gb_test"
obs["tod"] = todgb
obs["noise"] = focalplane.noise
obs["id"] = 2725
#obs["intervals"] = tod.subscans
#obs["site"] = site
obs["site_name"] = 'Teide Observatory'
obs["site_id"] = 128
obs["altitude"] = OT_ALTITUDE
#obs["weather"] = site.weather
obs["telescope_name"] = 'GroundBIRD'
obs["telescope_id"] = 0
obs["focalplane"] = focalplane.detector_data
obs["fpradius"] = focalplane.radius
obs["start_time"] = START_TIME
obs["season"] = 2020
obs["date"] = '2020-0101'
obs["MJD"] = 58849.034722
obs["rising"] = False
obs["mindist_sun"] = 86.36308327288089
obs["mindist_moon"] = 40.32976225720199
obs["el_sun"] = -18.68

# And append the observation to the list of observations

data.obs.append(obs)

## Pointing matrix

Here we translate the boresight quaternions into detector pointing (pixels numbers and Stokes weights).

In [ ]:
toast.todmap.OpPointingHpix(nside=args.nside, nest=True, mode=args.mode).exec(data)

Make a boolean hit map for diagnostics

In [ ]:
npix = 12 * args.nside ** 2
hitmap = np.zeros(npix)
for obs in data.obs:
    tod = obs["tod"]
    for det in tod.local_dets:
        pixels = tod.cache.reference("pixels_{}".format(det))
        hitmap[pixels] = 1
hitmap[hitmap == 0] = hp.UNSEEN
hp.mollview(hitmap, nest=True, title="all hit pixels", cbar=False)
hp.graticule(22.5, verbose=False)

In [ ]:
todgb._observer

## Sky signal

Create a synthetic Gaussian map to scan as input signal

In [ ]:
lmax = args.nside * 2
cls = np.zeros([4, lmax + 1])
cls[0] = 1e0
sim_map = hp.synfast(cls, args.nside, lmax=lmax, fwhm=np.radians(15), new=True)
plt.figure(figsize=[12, 8])
for i, m in enumerate(sim_map):
    hp.mollview(sim_map[i], cmap="coolwarm", title="Input signal {}".format("IQU"[i]), sub=[1, 3, 1+i])
hp.write_map("sim_map.fits", hp.reorder(sim_map, r2n=True), nest=True, overwrite=True)

Scan the sky signal

In [ ]:
full_name = "signal"
sky_name = "sky_signal"

# Clear any lingering sky signal from the buffers
toast.tod.OpCacheClear(full_name).exec(data)

distmap = toast.map.DistPixels(
    data,
    nnz=len(args.mode),
    dtype=np.float32,
)
distmap.read_healpix_fits("sim_map.fits")
toast.todmap.OpSimScan(distmap=distmap, out=full_name).exec(data)

# Copy the sky signal, just in case we need it later

toast.tod.OpCacheCopy(input=full_name, output=sky_name, force=True).exec(data)

## Noise

Simulate noise and make a copy of signal+noise in case we need it later

In [ ]:
copy_name = "signal_copy"

toast.tod.OpSimNoise(out=full_name, realization=0).exec(data)

toast.tod.OpCacheCopy(input=full_name, output=copy_name, force=True).exec(data)

## Your own operator here

Here we define an empty operator you can work with

Then we apply the operator to the data

Plot a short segment of the signal before and after the operator

## Make a map

Destripe the signal and make a map.  We use the nascent TOAST mapmaker because it can be run in serial mode without MPI.  The TOAST mapmaker is still significantly slower so production runs should used `libMadam`.

In [ ]:
len(tod._az)

In [ ]:
# Always begin mapmaking by copying the simulated signal.

destriped_name = "destriped"
toast.tod.OpCacheCopy(input=copy_name, output=destriped_name, force=True).exec(data)

mapmaker = toast.todmap.OpMapMaker(
    nside=args.nside,
    nnz=3,
    name=destriped_name,
    outdir=args.outdir,
    outprefix="toast_test_",
    baseline_length=10,
    iter_max=100,
    use_noise_prior=False,
)
mapmaker.exec(data)

Plot a segment of the timelines

In [ ]:
tod = data.obs[0]["tod"]
times = tod.local_times()

fig = plt.figure(figsize=[12, 8])
for idet, det in enumerate(tod.local_dets):
    sky = tod.local_signal(det, sky_name)
    full = tod.local_signal(det, copy_name)
    destriped = tod.local_signal(det, destriped_name)

    ind = slice(0, 1000)
    ax = fig.add_subplot(4, 4, 1 + idet)
    ax.set_title(det)
    ax.plot(times[ind], sky[ind], '.', label="sky", zorder=100)
    ax.plot(times[ind], full[ind] - sky[ind], '.', label="noise")
    ax.plot(times[ind], full[ind] - destriped[ind], '.', label="baselines")
ax.legend(bbox_to_anchor=(1.1, 1.00))
fig.subplots_adjust(hspace=0.6)


In [ ]:
PLEN=100000
plt.plot(tod.local_times()[:PLEN], tod.local_signal('0A', copy_name)[:PLEN])
diff = tod.local_signal('0A', copy_name)[:PLEN] - tod.local_signal('0A', destriped_name)[:PLEN]
plt.plot(tod.local_times()[:PLEN], diff)

In [ ]:
plt.figure(figsize=[16, 8])

hitmap = hp.read_map("maps/toast_test_hits.fits", verbose=False)
hitmap[hitmap == 0] = hp.UNSEEN
hp.mollview(hitmap, sub=[2, 2, 1], max=10000, title="hits")

binmap = hp.read_map("maps/toast_test_binned.fits", verbose=False)
binmap[binmap == 0] = hp.UNSEEN
hp.mollview(binmap, sub=[2, 2, 2], title="binned map", cmap="coolwarm")

# Fix the plotting range for input signal and the destriped map
amp = 3.0

destriped = hp.read_map("maps/toast_test_destriped.fits", verbose=False, field=0)
destriped[destriped == 0] = hp.UNSEEN
# Remove monopole
good = destriped != hp.UNSEEN
destriped[good] -= np.median(destriped[good])
hp.mollview(destriped, sub=[2, 2, 3], title="destriped map", cmap="coolwarm", min=-amp, max=amp)

inmap = hp.read_map("sim_map.fits", verbose=False, field=0)
inmap[hitmap == hp.UNSEEN] = hp.UNSEEN
hp.mollview(inmap, sub=[2, 2, 4], title="input map", cmap="coolwarm", min=-amp, max=amp)


In [ ]:
len(tod.local_signal('0A', sky_name))

In [ ]:
print(np.sum(hitmap[hitmap != hp.UNSEEN]) / 1400000.0)

In [ ]:
# Plot the white noise covariance

plt.figure(figsize=[12, 8])
wcov = hp.read_map("maps/toast_test_npp.fits", None)
wcov[:, wcov[0] == 0] = hp.UNSEEN
hp.mollview(wcov[0], sub=[3, 3, 1], title="II", cmap="coolwarm")
hp.mollview(wcov[1], sub=[3, 3, 2], title="IQ", cmap="coolwarm")
hp.mollview(wcov[2], sub=[3, 3, 3], title="IU", cmap="coolwarm")
hp.mollview(wcov[3], sub=[3, 3, 5], title="QQ", cmap="coolwarm")
hp.mollview(wcov[4], sub=[3, 3, 6], title="QU", cmap="coolwarm")
hp.mollview(wcov[5], sub=[3, 3, 9], title="UU", cmap="coolwarm")

## Filter & bin

A filter-and-bin mapmaker is easily created by combining TOAST filter operators and running the mapmaker without destriping:

In [ ]:
filtered_name = "filtered"

toast.tod.OpCacheCopy(input=copy_name, output=filtered_name, force=True).exec(data)

toast.tod.OpPolyFilter(order=5, name=filtered_name).exec(data)

mapmaker = toast.todmap.OpMapMaker(
    nside=args.nside,
    nnz=len(args.mode),
    name=filtered_name,
    outdir=args.outdir,
    outprefix="toast_test_filtered_",
    baseline_length=None,
    #flag_mask=0,
)
mapmaker.exec(data)

plt.figure(figsize=[16, 8])

binmap = hp.read_map("maps/toast_test_binned.fits", verbose=False)
binmap[binmap == 0] = hp.UNSEEN
hp.mollview(binmap, sub=[1, 3, 1], title="binned map", cmap="coolwarm")

filtered_map = hp.read_map("maps/toast_test_filtered_binned.fits", verbose=False)
filtered_map[filtered_map == 0] = hp.UNSEEN
hp.mollview(filtered_map, sub=[1, 3, 2], title="filtered map", cmap="coolwarm")

inmap = hp.read_map("sim_map.fits", verbose=False)
inmap[binmap == hp.UNSEEN] = hp.UNSEEN
hp.mollview(inmap, sub=[1, 3, 3], title="input map", cmap="coolwarm")

In [ ]:
highpass_name = 'highpass'

class TestOperator(toast.Operator):
    def __init__(self, name="signal", thr=0.1):
        """ Arguments:
        name(str) : Cache prefix to operate on
        """
        self._name = name
        self._thr = thr
    
    def exec(self, data):
        # We loop here over all local data but do nothing with it.
        for obs in data.obs:
            tod = obs["tod"]
            for det in tod.local_dets:
                signal = tod.local_signal(det, self._name)
                sig_fft = np.fft.fft(signal)
                sig_fre = np.fft.fftfreq(len(sig_fft), d=1/100)
                target = np.abs(sig_fre) < self._thr
                sig_fft[target] = 0
                sig_ifft = np.fft.ifft(sig_fft)
                signal[:] = np.abs(sig_ifft)*sig_ifft.real/np.abs(sig_ifft.real)

toast.tod.OpCacheCopy(input=copy_name, output=highpass_name, force=True).exec(data)
TestOperator(name=highpass_name, thr=0.1).exec(data)

In [ ]:
mapmaker = toast.todmap.OpMapMaker(
    nside=args.nside,
    nnz=len(args.mode),
    name=highpass_name,
    outdir=args.outdir,
    outprefix="toast_test_highpass_",
    baseline_length=None,
    #flag_mask=0,
)
mapmaker.exec(data)

plt.figure(figsize=[16, 8])

binmap = hp.read_map("maps/toast_test_binned.fits", verbose=False)
binmap[binmap == 0] = hp.UNSEEN
hp.mollview(binmap, sub=[1, 3, 1], title="binned map", cmap="coolwarm")

filtered_map = hp.read_map("maps/toast_test_highpass_binned.fits", verbose=False)
filtered_map[filtered_map == 0] = hp.UNSEEN
hp.mollview(filtered_map, sub=[1, 3, 2], title="filtered map", cmap="coolwarm")

inmap = hp.read_map("sim_map.fits", verbose=False)
inmap[binmap == hp.UNSEEN] = hp.UNSEEN
hp.mollview(inmap, sub=[1, 3, 3], title="input map", cmap="coolwarm")

In [ ]:
plt.figure(figsize=[16, 8])

hitmap = hp.read_map("maps/toast_test_hits.fits", verbose=False)
hitmap[hitmap == 0] = hp.UNSEEN
hp.mollview(hitmap, sub=[2, 2, 1], max=10000, title="hits")

binmap = hp.read_map("maps/toast_test_binned.fits", verbose=False)
binmap[binmap == 0] = hp.UNSEEN
hp.mollview(binmap, sub=[2, 2, 2], title="binned map", cmap="coolwarm")

# Fix the plotting range for input signal and the destriped map
amp = 3.0

destriped = hp.read_map("maps/toast_test_highpass_binned.fits", verbose=False, field=0)
destriped[destriped == 0] = hp.UNSEEN
# Remove monopole
good = destriped != hp.UNSEEN
destriped[good] -= np.median(destriped[good])
hp.mollview(destriped, sub=[2, 2, 3], title="Highpass map", cmap="coolwarm", min=-amp, max=amp)

inmap = hp.read_map("sim_map.fits", verbose=False, field=0)
inmap[hitmap == hp.UNSEEN] = hp.UNSEEN
hp.mollview(inmap, sub=[2, 2, 4], title="input map", cmap="coolwarm", min=-amp, max=amp)